# Projeto Final
## Análise de Texto de Fontes Desestruturadas e Web.

#### Integrantes
- Manuel Castanares
- Rafael Malcervelli 

### Objetivo

O projeto envolverá o uso das técnicas aprendidas em aula para conduzir, com autonomia, a criação de uma prova de conceito envolvendo a extração de dados de fontes desestruturadas e sua posterior análise.

### Introdução

No mercado de PC gamers, a escolha de componentes é uma decisão crítica que pode afetar significativamente o desempenho e a satisfação do usuário final. Dada a constante flutuação dos preços e a vasta gama de produtos disponíveis, consumidores e montadores de PCs enfrentam o desafio de identificar as melhores opções de compra. Este projeto propõe a utilização de técnicas de web scraping e análise de texto para extrair e analisar dados de múltiplos marketplaces, proporcionando uma ferramenta útil para a comparação de preços, disponibilidade de estoque e qualidade percebida dos componentes.

<p align=center>
    <img src=" ./imgs/pc_gamer_1.jpg" alt="PC GAMER" class="bg-primary mb-1" width="500px">
</p>

### Problema

A principal problemática abordada neste projeto é a dificuldade em obter uma visão clara e atualizada das melhores opções de compra para componentes de PCs gamers devido à volatilidade dos preços e à diversidade de produtos. Esta situação leva a decisões de compra que nem sempre são ótimas em termos de custo-benefício. Portanto, a necessidade de uma solução que agilize e refine o processo de escolha se faz essencial para consumidores e profissionais da área.

A situacao-problema que propomos aqui é a seguinte:<br><br>
Um jovem gostaria de montar um PC gamer que consiga rodar com facilidade os jogos atuais (gráfico avancados, demandando processamento de video) e jogos competitivos. Para isso, esse jovem ja tem em mente a maquina que gostaria de montar com todos os seus componentes e explicacoes dos mesmos, que seguem:

- Placa-mae: O 'coração' do computador, conecta todos os componentes, permitindo que eles se comuniquem. A escolha da placa-mãe influencia diretamente quais processadores e tipos de memória serão compatíveis. -> Placa escolhida para análise (**Asus TUF GAMING Z690-PLUS D4 ATX LGA 1700**). <br><br>
- CPU (Processador): O 'cérebro' do PC, crucial para o processamento de todas as tarefas. O desempenho do processador afeta diretamente a capacidade do sistema em executar jogos complexos sem travamentos. -> CPU escolhida para análise (**Intel Core i7 12700K 3.6 GHz 12-Core**).<br><br>
- GPU (Placa de vídeo): Fundamental para renderizar gráficos avançados em tempo real, a escolha da placa de vídeo é crucial para a experiência visual em jogos. -> GPU escolhida para análise (**Gigabyte GeForce RTX 4060 8 GB Gaming**).<br><br>
- Memória RAM: Responsável por armazenar os dados temporários do sistema, uma quantidade suficiente de RAM é vital para garantir uma operação suave e eficiente durante jogos intensivos. -> Memória escolhida para análise (**Kingston HyperX Fury Black Series (Preto) 8 GB (1x8 GB) DDR4-2666**).<br><br>
- Armazenamento (SSD): Discos de estado sólido (SSD) proporcionam tempos de carregamento mais rápidos para sistemas operacionais e jogos quando comparados aos discos rígidos tradicionais (HDD). -> SSD escolhido para análise (**SanDisk SSD Plus 1 TB 2.5"**).<br><br>
- Fonte de Alimentação: Supre a energia necessária para todos os componentes. Uma fonte de qualidade e com potência adequada é essencial para a estabilidade e longevidade do sistema. -> Fonte escolhida para análise (**Corsair CV550 550 W Certificado 80+ Bronze ATX**).<br><br>
- Fans (Ventoinhas): Cruciais para manter uma temperatura operacional adequada dentro do gabinete, evitando o superaquecimento dos componentes. -> Fan escolhido para análise (**Cooler Master Masterfan Mf120 Halo RGB**).<br><br>
- Cooler para CPU: Específico para o resfriamento do processador, é vital para manter o desempenho sem riscos de superaquecimento, especialmente em sessões prolongadas de jogos. -> Cooler escolhido para análise (**Deepcool AG400 BK**).<br><br>
- Gabinete: Estrutura do PC que irá segurar todas as pecas nos seus devidos lugares. Além da questao estetica, é sempre bom visar por gabinetes que apresentam um bom fluxo de ar dentro deles, permitindo uma maior circulacao e consequentemente, uma diminuicao na temperatura das pecas. -> Gabinete escolhido para análise (** **).

<p align=center>
    <img src=" ./imgs/pc_gamer_2.jpg" alt="PC GAMER 2" class="bg-primary mb-1" width="500px">
</p>